In [ ]:
#@title Import Data
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
import matplotlib.pyplot as plt
from sklearn.preprocessing import Normalizer
import nltk
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


# Pipeline Implementation

In [ ]:
!pip install stop-words

  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32894 sha256=8f66cfeaeb740c247cadc5e8641086707b37fbd1b1e079e81507dc1fa9477870
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop-words


In [ ]:
import time
import spacy
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer
from stop_words import get_stop_words

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


The ('to_dense', DenseTransformer()) step is included to convert sparse matrices to dense matrices because some classifiers (especially custom ones or those that expect dense input) may not work directly with sparse matrices, which are the output of TfidfVectorizer and CountVectorizer. In this case:

Why it's included: Scikit-Learn’s TfidfVectorizer and CountVectorizer produce sparse matrices to save memory and computation time, especially for large text datasets. However, certain estimators, such as BernoulliNB and some custom models, may require dense input to perform their calculations.

Why it wasn’t in your original code: If your original code worked without converting to dense format, it likely means that the classifiers in use (like Scikit-Learn’s BernoulliNB, DecisionTreeClassifier, and LogisticRegression) already support sparse matrices. However, when implementing custom pipelines or models, a dense transformer can sometimes be added to ensure compatibility, particularly when you’re unsure if the classifier requires dense data.

When it can be removed: If you’re only using Scikit-Learn’s classifiers that support sparse matrices, you can safely remove ('to_dense', DenseTransformer()). Scikit-Learn’s BernoulliNB, DecisionTreeClassifier, and LogisticRegression all support sparse input, so you should not need the dense conversion for these classifiers.

In [ ]:
'''# Function to create a pipeline with vectorizer and classifier
def create_pipeline(vectorizer, classifier):
    return Pipeline([
        ('lemmatize', LemmatizerTransformer()),
        ('vectorize', vectorizer),
        ('classify', classifier)
    ])'''

In [ ]:
# Load data
train_corpus = pd.read_csv('train.csv', encoding='cp720')
test_corpus = pd.read_csv('test.csv', encoding='cp720')

In [ ]:
train_corpus.head()

,body,subreddit
0,Hello dear Redditors! \nMy husband and I are ...,Paris
1,This city receives 50 millions of tourists per...,Paris
2,En vrai la circulation en vélo commence à être...,Paris
3,Nous avons effectué nos deux derniers déménage...,Paris
4,"If you couldn't find anything around there, I ...",Paris


In [ ]:
test_corpus.head()

,id,body
0,0,Anyone else feel like it?s getting to a point ...
1,1,Check out LeCypherX. Happens every last Thursd...
2,2,'That's mine bruv' - Steal your bike back.\n\n...
3,3,What might make it quite hard is that if this ...
4,4,# Upvote/Downvote reminder\n\nLike this image ...


In [ ]:
# Split data and define train indices
train_body = train_corpus['body']
subreddit = train_corpus['subreddit']
id_train = train_corpus.index  # Define train indices for tracking purposes
id_test = test_corpus['id']
test_body = test_corpus['body']

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Combine English and French stop words
english_stop_words = set(stopwords.words('english'))
french_stop_words = set(get_stop_words('french'))
combined_stop_words = english_stop_words.union(french_stop_words)

# Custom Preprocessing Transformer
class LemmatizerTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Remove stop words and lemmatize
        return [' '.join([lemmatizer.lemmatize(word.lower()) for word in word_tokenize(text) if word.lower() not in combined_stop_words]) for text in X]

# Transformer to convert sparse matrix to dense
class DenseTransformer(TransformerMixin):
    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.toarray()

# Function to create a pipeline with vectorizer, optional DenseTransformer, SVD, and classifier
def create_pipeline(vectorizer, svd, classifier, apply_dense_transform):
    steps = [
        ('lemmatize', LemmatizerTransformer()),  # Apply custom preprocessing
        ('vectorize', vectorizer)
    ]
    if apply_dense_transform:
        steps.append(('to_dense', DenseTransformer()))  # Conditionally add dense transformer
    if svd:  # Conditionally add SVD if specified
        steps.append(('svd', svd))
    steps.append(('classify', classifier))

    return Pipeline(steps)

# Classifier options
classifiers = {
    'Bernoulli Naive Bayes': BernoulliNB(alpha=1e-3),  # alpha is the Laplace smoothing parameter
    'Decision Tree': DecisionTreeClassifier(),
    'Logistic Regression': LogisticRegression(),
    'SVM': SVC(kernel='sigmoid'), # SVC(kernel='linear'), SVC(kernel='poly', degree=3), SVC(kernel='rbf', gamma='scale'), SVC(kernel='sigmoid')
    'Random Forest': RandomForestClassifier(n_estimators = 200),
    'XGBoost': XGBClassifier(eval_metric='mlogloss')
}

# Vectorizer options (stop words handled in preprocessing step)
vectorizers = {
    'TF-IDF': TfidfVectorizer(
        min_df=3, max_features=None, strip_accents='unicode', analyzer='word',
        token_pattern=r'\w{1,}', use_idf=True, smooth_idf=True, sublinear_tf=True,
        lowercase=True
    ),
    'Count Vectorizer': CountVectorizer(lowercase=True)
}

# Encode labels for XGBoost only
label_encoder = LabelEncoder()
subreddit_encoded = label_encoder.fit_transform(subreddit)

# Adjust the evaluate_pipeline function to include timing for each fold and overall runtime
def evaluate_pipeline(vectorizer_name, classifier_name, num_folds=5, svd_components=None, apply_dense_transform=True):
    vectorizer = vectorizers[vectorizer_name]
    classifier = classifiers[classifier_name]

    # Optional SVD step for dimensionality reduction
    svd = TruncatedSVD(n_components=svd_components) if svd_components else None

    # Encode labels for XGBoost if classifier is XGBoost
    y_data = subreddit_encoded if classifier_name == 'XGBoost' else subreddit

    # Create pipeline
    pipeline = create_pipeline(vectorizer, svd, classifier, apply_dense_transform)

    # Cross-validation with K-Fold
    kf = KFold(n_splits=num_folds, shuffle=True)
    fold_accuracies = []  # Store accuracy for each fold
    fold_times = []  # Store runtime for each fold

    print(f"Evaluating {classifier_name} with {vectorizer_name} across {num_folds} folds...\n")

    overall_start = time.time()  # Start time for the entire evaluation
    for fold, (train_index, val_index) in enumerate(kf.split(train_body), 1):
        fold_start = time.time()  # Start time for this fold

        # Split train and validation data for the current fold
        X_train, X_val = train_body.iloc[train_index], train_body.iloc[val_index]
        y_train, y_val = y_data[train_index], y_data[val_index]

        # Fit and evaluate the pipeline on the current fold
        pipeline.fit(X_train, y_train)
        fold_accuracy = pipeline.score(X_val, y_val)

        # End time for this fold and calculate fold time
        fold_end = time.time()
        fold_runtime = fold_end - fold_start
        fold_times.append(fold_runtime)

        # Print accuracy and runtime for the current fold and store the accuracy
        print(f"Fold {fold}: Accuracy = {fold_accuracy:.4f}, Runtime = {fold_runtime:.2f} seconds")
        fold_accuracies.append(fold_accuracy)

    # Calculate and print average accuracy and runtime
    avg_accuracy = np.mean(fold_accuracies)
    avg_runtime = np.mean(fold_times)
    overall_runtime = time.time() - overall_start
    print(f"\nAverage Accuracy ({classifier_name} with {vectorizer_name}): {avg_accuracy:.4f}")
    print(f"Average Fold Runtime: {avg_runtime:.2f} seconds")
    print(f"Total Runtime: {overall_runtime:.2f} seconds\n")

    # Train on the full dataset and make final predictions
    pipeline.fit(train_body, y_data)
    predicted_classes = pipeline.predict(test_body)

    # If classifier is XGBoost, convert numeric predictions back to original text labels
    if classifier_name == 'XGBoost':
        predicted_classes = label_encoder.inverse_transform(predicted_classes)

    # Store results in DataFrame
    results_df = pd.DataFrame({'id': id_test, 'subreddit': predicted_classes})

# Vectorizer: TF-IDF , Count Vectorizer
# Solver:     Bernoulli Naive Bayes, Decision Tree, Logistic Regression, SVM, Random Forest, XGBoost

#NOTE: Out of the models listed, only Logistic Regression and XGBoost use gradient-based optimization techniques

evaluate_pipeline('Count Vectorizer', 'Bernoulli Naive Bayes', num_folds=5, svd_components=None, apply_dense_transform=True)
#evaluate_pipeline('TF-IDF', 'SVM', num_folds=5, svd_components=600, apply_dense_transform=True)
evaluate_pipeline('TF-IDF', 'Random Forest', num_folds=5, svd_components=None, apply_dense_transform=True)
#evaluate_pipeline('TF-IDF', 'SVM', num_folds=5, svd_components=None, apply_dense_transform=False)
evaluate_pipeline('TF-IDF', 'SVM', num_folds=5, svd_components=500, apply_dense_transform=True)

Evaluating Bernoulli Naive Bayes with Count Vectorizer across 5 folds...

Fold 1: Accuracy = 0.6071, Runtime = 2.37 seconds
Fold 2: Accuracy = 0.6214, Runtime = 2.25 seconds
Fold 3: Accuracy = 0.6393, Runtime = 1.73 seconds
Fold 4: Accuracy = 0.6393, Runtime = 1.71 seconds
Fold 5: Accuracy = 0.6321, Runtime = 1.80 seconds

Average Accuracy (Bernoulli Naive Bayes with Count Vectorizer): 0.6279
Average Fold Runtime: 1.97 seconds
Total Runtime: 9.87 seconds

Evaluating Random Forest with TF-IDF across 5 folds...

Fold 1: Accuracy = 0.5964, Runtime = 7.07 seconds
Fold 2: Accuracy = 0.6679, Runtime = 3.98 seconds
Fold 3: Accuracy = 0.6000, Runtime = 5.18 seconds
Fold 4: Accuracy = 0.7000, Runtime = 3.34 seconds
Fold 5: Accuracy = 0.6107, Runtime = 3.24 seconds

Average Accuracy (Random Forest with TF-IDF): 0.6350
Average Fold Runtime: 4.56 seconds
Total Runtime: 22.81 seconds

Evaluating SVM with TF-IDF across 5 folds...

Fold 1: Accuracy = 0.6714, Runtime = 5.84 seconds
Fold 2: Accuracy = 

# Hyperparameter Turning (Grid Search)

**Bernoulli Naive Bayes**

In [ ]:
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV

# Define accuracy scorer using make_scorer
accuracy_scorer = make_scorer(accuracy_score)

# Define parameter grid for Bernoulli Naive Bayes with vectorizer, alpha, binarize, fit_prior, and dense transformation
param_grid = {
    'vectorize': [vectorizers['TF-IDF'], vectorizers['Count Vectorizer']],  # Different vectorizers
    'classify__alpha': [1e-6, 1e-4, 1e-2, 1e-1],  # Smoothing values for BernoulliNB
    'classify__binarize': [0.0, 1.0],  # Threshold for binarizing
    'classify__fit_prior': [True, False],  # Whether to learn class prior probabilities
    'to_dense__apply_dense': [True, False]  # Optionally apply DenseTransformer
}

# Conditional step in the pipeline based on the 'apply_dense' parameter
class ConditionalDenseTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, apply_dense):
        self.apply_dense = apply_dense

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return DenseTransformer().transform(X) if self.apply_dense else X

# Updated pipeline with conditional dense transformation
pipeline = Pipeline([
    ('lemmatize', LemmatizerTransformer()),
    ('vectorize', vectorizers['TF-IDF']),  # Placeholder, updated by grid search
    ('to_dense', ConditionalDenseTransformer(apply_dense=True)),  # Conditionally apply dense
    ('classify', BernoulliNB())  # Bernoulli Naive Bayes classifier
])

# Grid search setup with new parameter grid, using the custom accuracy scorer
grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=3,  # 3-fold cross-validation
    scoring=accuracy_scorer,
    refit=True,
    verbose=10
)

# Fit grid search to the data
grid_search.fit(train_body, subreddit)

# Display each combination's parameters and corresponding cross-validation results
print("Grid search results:\n")
for i, params in enumerate(grid_search.cv_results_['params']):
    mean_score = grid_search.cv_results_['mean_test_score'][i]
    std_score = grid_search.cv_results_['std_test_score'][i]
    print(f"Parameters: {params}")
    print(f"Mean cross-validation accuracy: {mean_score:.4f} (± {std_score:.4f})\n")

# Display the best parameters and cross-validation score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)

# Use the best model to predict on test_body
best_pipeline = grid_search.best_estimator_
test_predictions = best_pipeline.predict(test_body)

# Save the results
results_df = pd.DataFrame({'id': id_test, 'subreddit': test_predictions})
results_df.to_csv("final_predictions_with_best_params.csv", index=False)

Fitting 3 folds for each of 64 candidates, totalling 192 fits
[CV 1/3; 1/64] START classify__alpha=1e-06, classify__binarize=0.0, classify__fit_prior=True, to_dense__apply_dense=True, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')
[CV 1/3; 1/64] END classify__alpha=1e-06, classify__binarize=0.0, classify__fit_prior=True, to_dense__apply_dense=True, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}');, score=0.587 total time=   1.6s
[CV 2/3; 1/64] START classify__alpha=1e-06, classify__binarize=0.0, classify__fit_prior=True, to_dense__apply_dense=True, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')
[CV 2/3; 1/64] END classify__alpha=1e-06, classify__binarize=0.0, classify__fit_prior=True, to_dense__apply_dense=True, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode

**Decision Tree**

In [ ]:
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Define accuracy scorer using make_scorer
accuracy_scorer = make_scorer(accuracy_score)

# Define parameter grid for Decision Tree with vectorizer, max_depth, min_samples_split, min_samples_leaf, max_features, and dense transformation
param_grid = {
    'vectorize': [vectorizers['TF-IDF'], vectorizers['Count Vectorizer']],  # Different vectorizers
    'classify__max_depth': [None, 20],  # Maximum depth of the tree
    'classify__min_samples_split': [2, 10],  # Minimum number of samples required to split an internal node
    'classify__min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
    'classify__max_features': ['auto', 'sqrt', None],  # Number of features to consider for best split
    'classify__criterion': ['gini', 'entropy']  # Function to measure the quality of a split
}

# Conditional step in the pipeline based on the 'apply_dense' parameter
class ConditionalDenseTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, apply_dense=True):  # Set apply_dense=True by default
        self.apply_dense = apply_dense

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return DenseTransformer().transform(X) if self.apply_dense else X

# Updated pipeline with conditional dense transformation set to apply_dense=True
pipeline = Pipeline([
    ('lemmatize', LemmatizerTransformer()),
    ('vectorize', vectorizers['TF-IDF']),  # Placeholder, updated by grid search
    ('to_dense', ConditionalDenseTransformer(apply_dense=True)),  # Always apply dense transformation
    ('classify', DecisionTreeClassifier())  # Decision Tree classifier
])

# Grid search setup with new parameter grid, using the custom accuracy scorer
grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=2,  # 2-fold cross-validation for quick testing, adjust as needed
    scoring=accuracy_scorer,
    refit=True,
    verbose=10
)

# Fit grid search to the data
grid_search.fit(train_body, subreddit)

# Display each combination's parameters and corresponding cross-validation results
print("Grid search results:\n")
for i, params in enumerate(grid_search.cv_results_['params']):
    mean_score = grid_search.cv_results_['mean_test_score'][i]
    std_score = grid_search.cv_results_['std_test_score'][i]
    print(f"Parameters: {params}")
    print(f"Mean cross-validation accuracy: {mean_score:.4f} (± {std_score:.4f})\n")

# Display the best parameters and cross-validation score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)

# Use the best model to predict on test_body
best_pipeline = grid_search.best_estimator_
test_predictions = best_pipeline.predict(test_body)

# Save the results
results_df = pd.DataFrame({'id': id_test, 'subreddit': test_predictions})
results_df.to_csv("final_predictions_with_best_params_decision_tree.csv", index=False)
print("\nPredictions saved to final_predictions_with_best_params_decision_tree.csv")


Fitting 2 folds for each of 144 candidates, totalling 288 fits
[CV 1/2; 1/144] START classify__criterion=gini, classify__max_depth=None, classify__max_features=auto, classify__min_samples_leaf=1, classify__min_samples_split=2, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')
[CV 1/2; 1/144] END classify__criterion=gini, classify__max_depth=None, classify__max_features=auto, classify__min_samples_leaf=1, classify__min_samples_split=2, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}');, score=nan total time=   0.8s
[CV 2/2; 1/144] START classify__criterion=gini, classify__max_depth=None, classify__max_features=auto, classify__min_samples_leaf=1, classify__min_samples_split=2, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')
[CV 2/2; 1/144] END classify__criterion=gini, classif

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
96 fits failed out of a total of 288.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
96 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 473, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/usr/local/lib/pyt

Grid search results:

Parameters: {'classify__criterion': 'gini', 'classify__max_depth': None, 'classify__max_features': 'auto', 'classify__min_samples_leaf': 1, 'classify__min_samples_split': 2, 'vectorize': TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')}
Mean cross-validation accuracy: nan (± nan)

Parameters: {'classify__criterion': 'gini', 'classify__max_depth': None, 'classify__max_features': 'auto', 'classify__min_samples_leaf': 1, 'classify__min_samples_split': 2, 'vectorize': CountVectorizer()}
Mean cross-validation accuracy: nan (± nan)

Parameters: {'classify__criterion': 'gini', 'classify__max_depth': None, 'classify__max_features': 'auto', 'classify__min_samples_leaf': 1, 'classify__min_samples_split': 10, 'vectorize': TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')}
Mean cross-validation accuracy: nan (± nan)

Parameters: {'classify__criterion': 'g

**Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV

# Define accuracy scorer using make_scorer
accuracy_scorer = make_scorer(accuracy_score)

# Define parameter grid for Logistic Regression with vectorizer, regularization (C), and solver
param_grid = {
    'vectorize': [vectorizers['TF-IDF'], vectorizers['Count Vectorizer']],  # Different vectorizers
    'classify__C': [0.01, 0.1, 1.0, 10],  # Regularization strength (inverse of alpha)
    'classify__penalty': ['l2', 'none'],  # Regularization penalty
    'classify__solver': ['lbfgs', 'saga'],  # Solvers compatible with L2 penalty and dense data
}

# Conditional step in the pipeline based on the 'apply_dense' parameter, set to False
class ConditionalDenseTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, apply_dense=False):
        self.apply_dense = apply_dense

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return DenseTransformer().transform(X) if self.apply_dense else X

# Updated pipeline with dense transformation set to False
pipeline = Pipeline([
    ('lemmatize', LemmatizerTransformer()),
    ('vectorize', vectorizers['TF-IDF']),  # Placeholder, updated by grid search
    ('to_dense', ConditionalDenseTransformer(apply_dense=False)),  # Set dense transformation to False
    ('classify', LogisticRegression(max_iter=1000))  # Logistic Regression classifier with increased iterations
])

# Grid search setup with the updated parameter grid and custom accuracy scorer
grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=2,  # Adjust as needed
    scoring=accuracy_scorer,
    refit=True,
    verbose=10  # Higher verbosity for detailed output
)

# Fit grid search to the data
grid_search.fit(train_body, subreddit)

# Display each combination's parameters and corresponding cross-validation results
print("Grid search results:\n")
for i, params in enumerate(grid_search.cv_results_['params']):
    mean_score = grid_search.cv_results_['mean_test_score'][i]
    std_score = grid_search.cv_results_['std_test_score'][i]
    print(f"Parameters: {params}")
    print(f"Mean cross-validation accuracy: {mean_score:.4f} (± {std_score:.4f})\n")

# Display the best parameters and cross-validation score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)

# Use the best model to predict on test_body
best_pipeline = grid_search.best_estimator_
test_predictions = best_pipeline.predict(test_body)

# Save the results
results_df = pd.DataFrame({'id': id_test, 'subreddit': test_predictions})
results_df.to_csv("final_predictions_with_best_params_logistic_regression.csv", index=False)
print("\nPredictions saved to final_predictions_with_best_params_logistic_regression.csv")


Fitting 2 folds for each of 32 candidates, totalling 64 fits
[CV 1/2; 1/32] START classify__C=0.01, classify__penalty=l2, classify__solver=lbfgs, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')
[CV 1/2; 1/32] END classify__C=0.01, classify__penalty=l2, classify__solver=lbfgs, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}');, score=0.571 total time=   1.5s
[CV 2/2; 1/32] START classify__C=0.01, classify__penalty=l2, classify__solver=lbfgs, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')
[CV 2/2; 1/32] END classify__C=0.01, classify__penalty=l2, classify__solver=lbfgs, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}');, score=0.651 total time=   1.5s
[CV 1/2; 2/32] START classify__C=0.01, classify__pena

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
32 fits failed out of a total of 64.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
32 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 473, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/usr/local/lib/pyth

Grid search results:

Parameters: {'classify__C': 0.01, 'classify__penalty': 'l2', 'classify__solver': 'lbfgs', 'vectorize': TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')}
Mean cross-validation accuracy: 0.6114 (± 0.0400)

Parameters: {'classify__C': 0.01, 'classify__penalty': 'l2', 'classify__solver': 'lbfgs', 'vectorize': CountVectorizer()}
Mean cross-validation accuracy: 0.6307 (± 0.0350)

Parameters: {'classify__C': 0.01, 'classify__penalty': 'l2', 'classify__solver': 'saga', 'vectorize': TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')}
Mean cross-validation accuracy: 0.5929 (± 0.0271)

Parameters: {'classify__C': 0.01, 'classify__penalty': 'l2', 'classify__solver': 'saga', 'vectorize': CountVectorizer()}
Mean cross-validation accuracy: 0.6493 (± 0.0364)

Parameters: {'classify__C': 0.01, 'classify__penalty': 'none', 'classify__solver': 'lbfgs', 'vectorize

**SVD + SVM**

In [ ]:
from sklearn.svm import SVC
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV

# Define accuracy scorer using make_scorer
accuracy_scorer = make_scorer(accuracy_score)

# Define parameter grid for SVM with vectorizer, SVD components, C, kernel, and gamma
param_grid = {
    'vectorize': [vectorizers['TF-IDF'], vectorizers['Count Vectorizer']],  # Different vectorizers
    'svd__n_components': [200, 600],  # SVD components
    'classify__C': [0.01, 1, 10],  # Regularization parameter
    'classify__kernel': ['linear', 'sigmoid'],  # Different kernels
    'classify__gamma': ['scale', 'auto']  # Kernel coefficient
}

# Conditional step in the pipeline based on the 'apply_dense' parameter, set to False
class ConditionalDenseTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, apply_dense=False):
        self.apply_dense = apply_dense

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return DenseTransformer().transform(X) if self.apply_dense else X

# Updated pipeline with dense transformation set to False and SVD step
pipeline = Pipeline([
    ('lemmatize', LemmatizerTransformer()),
    ('vectorize', vectorizers['TF-IDF']),  # Placeholder, updated by grid search
    ('to_dense', ConditionalDenseTransformer(apply_dense=False)),  # Set dense transformation to False
    ('svd', TruncatedSVD()),  # Placeholder for SVD, parameter updated by grid search
    ('classify', SVC())  # SVM classifier
])

# Grid search setup with the updated parameter grid and custom accuracy scorer
grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=2,  # Adjust as needed
    scoring=accuracy_scorer,
    refit=True,
    verbose=10  # Higher verbosity for detailed output
)

# Fit grid search to the data
grid_search.fit(train_body, subreddit)

# Display each combination's parameters and corresponding cross-validation results
print("Grid search results:\n")
for i, params in enumerate(grid_search.cv_results_['params']):
    mean_score = grid_search.cv_results_['mean_test_score'][i]
    std_score = grid_search.cv_results_['std_test_score'][i]
    print(f"Parameters: {params}")
    print(f"Mean cross-validation accuracy: {mean_score:.4f} (± {std_score:.4f})\n")

# Display the best parameters and cross-validation score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)

# Use the best model to predict on test_body
best_pipeline = grid_search.best_estimator_
test_predictions = best_pipeline.predict(test_body)

# Save the results
results_df = pd.DataFrame({'id': id_test, 'subreddit': test_predictions})
results_df.to_csv("final_predictions_with_best_params_svm.csv", index=False)
print("\nPredictions saved to final_predictions_with_best_params_svm.csv")


Fitting 2 folds for each of 48 candidates, totalling 96 fits
[CV 1/2; 1/48] START classify__C=0.01, classify__gamma=scale, classify__kernel=linear, svd__n_components=200, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')
[CV 1/2; 1/48] END classify__C=0.01, classify__gamma=scale, classify__kernel=linear, svd__n_components=200, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}');, score=0.464 total time=   4.1s
[CV 2/2; 1/48] START classify__C=0.01, classify__gamma=scale, classify__kernel=linear, svd__n_components=200, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')
[CV 2/2; 1/48] END classify__C=0.01, classify__gamma=scale, classify__kernel=linear, svd__n_components=200, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_patte

**Random Forest**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV

# Define accuracy scorer
accuracy_scorer = make_scorer(accuracy_score)

# Define parameter grid for RandomForestClassifier with different vectorizers and hyperparameters
param_grid = {
    'vectorize': [vectorizers['TF-IDF'], vectorizers['Count Vectorizer']],  # Different vectorizers
    'classify__n_estimators': [100, 200],  # Number of trees in the forest
    'classify__max_depth': [None, 10, 20],  # Maximum depth of the tree
    'classify__min_samples_split': [2, 5],  # Minimum samples required to split an internal node
    'classify__min_samples_leaf': [1, 2]  # Minimum samples required to be at a leaf node
}

# Define a pipeline with RandomForestClassifier and no SVD
pipeline = Pipeline([
    ('lemmatize', LemmatizerTransformer()),
    ('vectorize', vectorizers['TF-IDF']),  # Placeholder; updated by grid search
    ('classify', RandomForestClassifier(random_state=42))  # Random Forest classifier
])

# Initialize GridSearchCV with the pipeline and parameter grid
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=3,  # 3-fold cross-validation
    scoring=accuracy_scorer,
    refit=True,
    verbose=10
)

# Fit grid search to the data
grid_search.fit(train_body, subreddit)

# Display grid search results
print("Grid search results:\n")
for i, params in enumerate(grid_search.cv_results_['params']):
    mean_score = grid_search.cv_results_['mean_test_score'][i]
    std_score = grid_search.cv_results_['std_test_score'][i]
    print(f"Parameters: {params}")
    print(f"Mean cross-validation accuracy: {mean_score:.4f} (± {std_score:.4f})\n")

# Display the best parameters and cross-validation score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)

# Predict on test_body with the best model
best_pipeline = grid_search.best_estimator_
test_predictions = best_pipeline.predict(test_body)

# Save predictions
results_df = pd.DataFrame({'id': id_test, 'subreddit': test_predictions})
results_df.to_csv("final_predictions_with_best_params_rf.csv", index=False)
print("\nPredictions saved to final_predictions_with_best_params_rf.csv")


Fitting 3 folds for each of 48 candidates, totalling 144 fits
[CV 1/3; 1/48] START classify__max_depth=None, classify__min_samples_leaf=1, classify__min_samples_split=2, classify__n_estimators=100, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')
[CV 1/3; 1/48] END classify__max_depth=None, classify__min_samples_leaf=1, classify__min_samples_split=2, classify__n_estimators=100, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}');, score=0.653 total time=   2.3s
[CV 2/3; 1/48] START classify__max_depth=None, classify__min_samples_leaf=1, classify__min_samples_split=2, classify__n_estimators=100, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')
[CV 2/3; 1/48] END classify__max_depth=None, classify__min_samples_leaf=1, classify__min_samples_split=2, classify__n_estimators=100, ve

**XGBoost**

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV

# Define accuracy scorer
accuracy_scorer = make_scorer(accuracy_score)

# Encode labels for XGBoost only
label_encoder = LabelEncoder()
subreddit_encoded = label_encoder.fit_transform(subreddit)  # Use these labels for training

# Define parameter grid for XGBoost with vectorizer options
param_grid = {
    'vectorize': [vectorizers['TF-IDF'], vectorizers['Count Vectorizer']],
    'classify__learning_rate': [0.001, 0.01, 0.1],         # Learning rate
    'classify__n_estimators': [100, 200],                # Number of boosting rounds
    'classify__max_depth': [3, 5],                       # Max depth of the trees
    'classify__subsample': [0.7, 1.0]                    # Subsample ratio of the training instance
}

# Updated pipeline
pipeline = Pipeline([
    ('lemmatize', LemmatizerTransformer()),
    ('vectorize', vectorizers['TF-IDF']),                # Placeholder; updated by grid search
    ('classify', XGBClassifier(eval_metric='mlogloss'))
])

# Grid search setup with XGBoost
grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=5,                    # 5-fold cross-validation
    scoring=accuracy_scorer,
    refit=True,
    verbose=10
)

# Fit grid search to the encoded labels
grid_search.fit(train_body, subreddit_encoded)

# Display grid search results
print("Grid search results:\n")
for i, params in enumerate(grid_search.cv_results_['params']):
    mean_score = grid_search.cv_results_['mean_test_score'][i]
    std_score = grid_search.cv_results_['std_test_score'][i]
    print(f"Parameters: {params}")
    print(f"Mean cross-validation accuracy: {mean_score:.4f} (± {std_score:.4f})\n")

# Display the best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation accuracy:", grid_search.best_score_)

# Predict on test_body with the best model
best_pipeline = grid_search.best_estimator_
test_predictions = best_pipeline.predict(test_body)

# Convert numeric predictions back to original labels for final output
test_predictions_labels = label_encoder.inverse_transform(test_predictions)

# Save predictions
results_df = pd.DataFrame({'id': id_test, 'subreddit': test_predictions_labels})
results_df.to_csv("final_predictions_with_best_params_xgboost.csv", index=False)
print("\nPredictions saved to final_predictions_with_best_params_xgboost.csv")


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV 1/5; 1/48] START classify__learning_rate=0.001, classify__max_depth=3, classify__n_estimators=100, classify__subsample=0.7, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')
[CV 1/5; 1/48] END classify__learning_rate=0.001, classify__max_depth=3, classify__n_estimators=100, classify__subsample=0.7, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}');, score=0.571 total time=   6.4s
[CV 2/5; 1/48] START classify__learning_rate=0.001, classify__max_depth=3, classify__n_estimators=100, classify__subsample=0.7, vectorize=TfidfVectorizer(min_df=3, strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w{1,}')
[CV 2/5; 1/48] END classify__learning_rate=0.001, classify__max_depth=3, classify__n_estimators=100, classify__subsample=0.7, vectorize=TfidfVectorizer(min_df=3

# Ensemble Learning: Majority Vote

The voting process works on a per-instance basis for each test instance, combining predictions from each model in the ensemble to decide on the final prediction. Here’s how the process works step-by-step:


1.   Individual Model Predictions: For each test instance, each model in the ensemble makes its own prediction independently.
2.   Collect Predictions for Each Instance: After each model predicts for the test instance, the ensemble gathers the predictions from all models.
3. Voting Mechanism:


  *   In hard voting, the most common prediction (the “majority vote”) is chosen as the final prediction for that instance.
  *   For soft voting (only applicable if models can output probabilities), each model assigns probabilities to each class, and the ensemble sums these probabilities across models. The class with the highest combined probability is chosen as the final prediction.

4. Repeat for Each Test Instance: This voting process happens independently for each test instance, so the predictions are made one by one for each instance, resulting in a final array of predictions.

So, to summarize, voting happens individually for each instance by aggregating predictions from all models and selecting the final class based on the chosen voting strategy.









The difference between hard and soft voting lies in how the final prediction is made in an ensemble of classifiers:

1. Hard Voting:
*   In hard voting, each base classifier makes a prediction (a "vote") for each sample, and the ensemble's final prediction is the label that receives the majority of the votes.

*   It’s essentially a majority rule: if most classifiers predict one label, that becomes the final prediction.

*   Hard voting works well when individual classifiers perform well and agree often.

\\

2. Soft Voting:
*   In soft voting, each classifier provides a probability estimate for each class label rather than a fixed prediction.
*   The ensemble takes the average of these probabilities for each class and predicts the class with the highest average probability.
*   Soft voting generally performs better when the base classifiers are well-calibrated (meaning their probability estimates reflect their confidence) because it considers the confidence of each prediction rather than treating all predictions equally.
*   This approach requires classifiers that can produce probability estimates (e.g., predict_proba method), such as Logistic Regression, Random Forest, and SVM when probability=True is set.

In general, soft voting can outperform hard voting because it incorporates the degree of confidence each model has in its prediction, making it less sensitive to any individual model’s mistakes.

In [ ]:
import time
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np

# Initialize label encoder for XGBoost
label_encoder = LabelEncoder()
subreddit_encoded = label_encoder.fit_transform(subreddit)

# Define vectorizers
tfidf_vectorizer = TfidfVectorizer(
    min_df=3, max_features=None, strip_accents='unicode', analyzer='word',
    token_pattern=r'\w{1,}', use_idf=True, smooth_idf=True, sublinear_tf=True,
    lowercase=True
)
count_vectorizer = CountVectorizer(lowercase=True)

# Set best parameters for each model (excluding vectorizer)
best_params = {
    'BernoulliNB': {'alpha': 0.01, 'binarize': 0.0, 'fit_prior': True},
    'DecisionTree': {'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': None, 'criterion': 'gini'},
    'LogisticRegression': {'C': 0.1, 'penalty': 'l2', 'solver': 'saga'},
    'SVM': {'C': 10.0, 'kernel': 'linear', 'gamma': 'scale', 'probability': True},  # with SVD
    'RandomForest': {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 5, 'min_samples_leaf': 2},
    'XGBoost': {'learning_rate': 0.1, 'n_estimators': 400, 'max_depth': 3, 'subsample': 1.0}
}

# Define the function to evaluate the pipeline
def evaluate_pipeline(voting_type='hard'):
    # Base models with best parameters, specify vectorizer within each model pipeline
    models = [
        ('bernoulli_nb', Pipeline([
            ('vectorize', count_vectorizer),  # Use Count Vectorizer for BernoulliNB
            ('classify', BernoulliNB(**best_params['BernoulliNB']))
        ])),
        ('decision_tree', Pipeline([
            ('vectorize', tfidf_vectorizer),  # Use TF-IDF for Decision Tree
            ('classify', DecisionTreeClassifier(**best_params['DecisionTree']))
        ])),
        ('logistic_reg', Pipeline([
            ('vectorize', count_vectorizer),  # Use Count Vectorizer for Logistic Regression
            ('classify', LogisticRegression(**best_params['LogisticRegression'], max_iter=2000))
        ])),
        ('svm', Pipeline([
            ('vectorize', tfidf_vectorizer),  # Use TF-IDF for SVM
            ('svd', TruncatedSVD(n_components=1000)),  # SVD for dimensionality reduction with SVM
            ('classify', SVC(**best_params['SVM']))
        ])),
        ('random_forest', Pipeline([
            ('vectorize', tfidf_vectorizer),  # Use TF-IDF for Random Forest
            ('classify', RandomForestClassifier(**best_params['RandomForest']))
        ])),
        ('xgboost', Pipeline([
            ('vectorize', count_vectorizer),  # Use Count Vectorizer for XGBoost
            ('classify', XGBClassifier(**best_params['XGBoost'], eval_metric='mlogloss'))
        ]))
    ]

    # Voting classifier with specified voting type
    voting_clf = VotingClassifier(estimators=models, voting=voting_type)

    # Set up K-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    fold_accuracies = []
    fold_times = []

    print(f"Evaluating with {voting_type} voting across 5 folds...\n")
    overall_start = time.time()

    # Loop through each fold
    for fold, (train_index, val_index) in enumerate(kf.split(train_body), 1):
        fold_start = time.time()

        # Split the data
        X_train, X_val = train_body.iloc[train_index], train_body.iloc[val_index]
        y_train, y_val = subreddit_encoded[train_index], subreddit_encoded[val_index]

        # Fit the voting classifier
        voting_clf.fit(X_train, y_train)

        # Predict and evaluate
        val_predictions = voting_clf.predict(X_val)
        fold_accuracy = accuracy_score(y_val, val_predictions)

        # Record time and accuracy for the fold
        fold_end = time.time()
        fold_runtime = fold_end - fold_start
        fold_accuracies.append(fold_accuracy)
        fold_times.append(fold_runtime)

        print(f"Fold {fold}: Accuracy = {fold_accuracy:.4f}, Runtime = {fold_runtime:.2f} seconds")

    # Calculate average metrics
    avg_accuracy = np.mean(fold_accuracies)
    avg_runtime = np.mean(fold_times)
    overall_runtime = time.time() - overall_start

    print(f"\nAverage Accuracy ({voting_type} voting): {avg_accuracy:.4f}")
    print(f"Average Fold Runtime: {avg_runtime:.2f} seconds")
    print(f"Total Runtime: {overall_runtime:.2f} seconds\n")

# Run the function for both voting types
evaluate_pipeline(voting_type='hard')
evaluate_pipeline(voting_type='soft')


Evaluating with hard voting across 5 folds...

Fold 1: Accuracy = 0.6821, Runtime = 21.08 seconds
Fold 2: Accuracy = 0.6536, Runtime = 26.25 seconds
Fold 3: Accuracy = 0.6643, Runtime = 24.24 seconds
Fold 4: Accuracy = 0.7036, Runtime = 20.28 seconds
Fold 5: Accuracy = 0.6821, Runtime = 24.02 seconds

Average Accuracy (hard voting): 0.6771
Average Fold Runtime: 23.18 seconds
Total Runtime: 115.89 seconds

Evaluating with soft voting across 5 folds...

Fold 1: Accuracy = 0.6750, Runtime = 20.63 seconds
Fold 2: Accuracy = 0.6893, Runtime = 23.08 seconds
Fold 3: Accuracy = 0.7214, Runtime = 25.32 seconds
Fold 4: Accuracy = 0.7107, Runtime = 20.09 seconds
Fold 5: Accuracy = 0.6714, Runtime = 24.69 seconds

Average Accuracy (soft voting): 0.6936
Average Fold Runtime: 22.76 seconds
Total Runtime: 113.83 seconds



# Deep Neural Networks

## Dense Network (FFNN)

In [ ]:
# to import tensorflow
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
import time
from tensorflow.keras.layers import Input

# Define vectorizers
tfidf_vectorizer = TfidfVectorizer(
    min_df=3, max_features=None, strip_accents='unicode', analyzer='word',
    token_pattern=r'\w{1,}', use_idf=True, smooth_idf=True, sublinear_tf=True,
    lowercase=True
)
count_vectorizer = CountVectorizer(lowercase=True, max_features=None)

# Choose vectorizer
vectorizer = tfidf_vectorizer  # You can switch to count_vectorizer if desired

# Vectorize the text data
X = vectorizer.fit_transform(train_body).toarray()
X_test = vectorizer.transform(test_body).toarray()

# Encode target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(subreddit)

# Define the dense neural network model
def create_dnn(input_dim, num_classes):
    model = Sequential([
        Input(shape=(input_dim,)),  # Define input shape with Input layer
        Dense(512, activation='relu'),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(num_classes, activation='softmax')  # Output layer with softmax for multi-class classification
    ])

    # Compile the model with Adam optimizer and categorical crossentropy loss for multi-class classification
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []
fold_times = []

print("Performing K-fold cross-validation on the dense neural network...\n")
overall_start = time.time()

for fold, (train_index, val_index) in enumerate(kf.split(X), 1):
    fold_start = time.time()

    # Split data into training and validation sets for this fold
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Initialize the model
    model = create_dnn(input_dim=X_train.shape[1], num_classes=len(np.unique(y)))

    # Early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

    # Train the model
    model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1
    )

    # Evaluate on validation data
    val_predictions = model.predict(X_val)
    val_predictions = np.argmax(val_predictions, axis=1)
    fold_accuracy = accuracy_score(y_val, val_predictions)

    # Record accuracy and runtime
    fold_end = time.time()
    fold_runtime = fold_end - fold_start
    fold_accuracies.append(fold_accuracy)
    fold_times.append(fold_runtime)

    print(f"Fold {fold}: Accuracy = {fold_accuracy:.4f}, Runtime = {fold_runtime:.2f} seconds")

# Calculate average metrics
avg_accuracy = np.mean(fold_accuracies)
avg_runtime = np.mean(fold_times)
overall_runtime = time.time() - overall_start

print(f"\nAverage Accuracy: {avg_accuracy:.4f}")
print(f"Average Fold Runtime: {avg_runtime:.2f} seconds")
print(f"Total Runtime: {overall_runtime:.2f} seconds\n")

# Final training on all training data for test prediction
final_model = create_dnn(input_dim=X.shape[1], num_classes=len(np.unique(y)))
final_model.fit(
    X, y,
    epochs=30,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=0
)

# Predict on the test data
test_predictions = final_model.predict(X_test)
test_predictions_labels = label_encoder.inverse_transform(np.argmax(test_predictions, axis=1))

# Save final test predictions
results_test_df = pd.DataFrame({'id': id_test, 'subreddit': test_predictions_labels})
results_test_df.to_csv("final_predictions_dnn_test.csv", index=False)
print("\nFinal test predictions saved to final_predictions_dnn_test.csv")


Performing K-fold cross-validation on the dense neural network...

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 10s 63ms/step - accuracy: 0.2525 - loss: 1.3843 - val_accuracy: 0.2500 - val_loss: 1.3793
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.3601 - loss: 1.3695 - val_accuracy: 0.3000 - val_loss: 1.3612
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3780 - loss: 1.3410 - val_accuracy: 0.3929 - val_loss: 1.3228
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4618 - loss: 1.2856 - val_accuracy: 0.4321 - val_loss: 1.2634
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5186 - loss: 1.2131 - val_accuracy: 0.4321 - val_loss: 1.2098
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5635 - loss: 1.1460 - val_accuracy: 0.4821 - val_loss: 1.1653
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6484 - loss: 1.0531 - val_accuracy: 0.5143 - val_loss: 1.1229
Epoch 8/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 

**Hyperparameter Tuning**

In [ ]:
from sklearn.model_selection import ParameterGrid
# Define vectorizers
tfidf_vectorizer = TfidfVectorizer(
    min_df=2, max_features=None, strip_accents='unicode', analyzer='word',
    token_pattern=r'\w{1,}', use_idf=True, smooth_idf=True, sublinear_tf=True,
    lowercase=True
)
count_vectorizer = CountVectorizer(lowercase=True, max_features=None)

# Choose vectorizer
vectorizer = tfidf_vectorizer  # You can switch to count_vectorizer if desired

# Vectorize the text data
X = vectorizer.fit_transform(train_body).toarray()
X_test = vectorizer.transform(test_body).toarray()
# Converts the sparse matrix from TF-IDF vectorization into a dense array (a regular NumPy array).
# In case of using Sklearn, we can skip it because Sklearn works well with sparse matrix. But TensorFlow does not!

# Encode target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(subreddit)

# Define the function to create the DNN with customizable hyperparameters
def create_dnn(num_classes, layer_sizes, dropout_rate):
    model = Sequential([
        Input(shape=(X.shape[1],)),
        Dense(layer_sizes[0], activation='relu'),
        Dropout(dropout_rate),
        Dense(layer_sizes[1], activation='relu'),
        Dropout(dropout_rate),
        Dense(layer_sizes[2], activation='relu'),
        Dropout(dropout_rate),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Best parameters: {'batch_size': 32, 'dropout_rate': 0.4, 'layer_sizes': (256, 128, 64), 'learning_rate': 0.0001}
# Hyperparameter grid
param_grid = {
    'learning_rate': [0.001, 0.0001],
    'layer_sizes': [(512, 256, 128), (256, 128, 64), (128, 64, 32)],
    'dropout_rate': [0.2, 0.5],
    'batch_size': [32, 64]
}

# Initialize variables to store the best hyperparameters and accuracy
best_accuracy = 0
best_params = None

print("Hyperparameter tuning...\n")

# Iterate over each combination of hyperparameters
for params in ParameterGrid(param_grid):
    learning_rate = params['learning_rate']
    layer_sizes = params['layer_sizes']
    dropout_rate = params['dropout_rate']
    batch_size = params['batch_size']

    print(f"Testing parameters: {params}")
    kf = KFold(n_splits=3, shuffle=True, random_state=42)
    fold_accuracies = []

    for fold, (train_index, val_index) in enumerate(kf.split(X), 1):
        # Split data into training and validation sets for this fold
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        # Initialize the model with the current set of hyperparameters
        model = create_dnn(num_classes=len(np.unique(y)), layer_sizes=layer_sizes, dropout_rate=dropout_rate)
        model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        # Early stopping to prevent overfitting
        early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

        # Train the model
        model.fit(
            X_train, y_train,
            epochs=50,
            batch_size=batch_size,
            validation_data=(X_val, y_val),
            callbacks=[early_stopping],
            verbose=0
        )

        # Evaluate on validation data
        val_predictions = model.predict(X_val)
        val_predictions = np.argmax(val_predictions, axis=1)
        fold_accuracy = accuracy_score(y_val, val_predictions)
        fold_accuracies.append(fold_accuracy)

    # Calculate average accuracy for this parameter set
    avg_accuracy = np.mean(fold_accuracies)
    print(f"Parameters: {params}, Average Accuracy = {avg_accuracy:.4f}")

    # Update best parameters based on accuracy
    if avg_accuracy > best_accuracy:
        best_accuracy = avg_accuracy
        best_params = params

print(f"\nBest parameters: {best_params} with average accuracy: {best_accuracy:.4f}\n")

# Final training on all training data for test prediction with the best parameters
final_model = create_dnn(
    num_classes=len(np.unique(y)),
    layer_sizes=best_params['layer_sizes'],
    dropout_rate=best_params['dropout_rate']
)
final_model.compile(optimizer=Adam(learning_rate=best_params['learning_rate']), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Early stopping for final model training
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model on the full training dataset with the best parameters
final_model.fit(
    X, y,
    epochs=50,
    batch_size=best_params['batch_size'],
    validation_split=0.2,
    callbacks=[early_stopping],
    verbose=0
)

# Predict on the test data
test_predictions = final_model.predict(X_test)
test_predictions_labels = np.argmax(test_predictions, axis=1)  # Get the predicted class indices

# Calculate and print accuracy on test data (only if y_test is available)
# Uncomment the following lines if you have y_test
# test_accuracy = accuracy_score(y_test, test_predictions_labels)
# print(f"\nTest Accuracy: {test_accuracy:.4f}")

# Convert predictions back to original labels if needed
test_predictions_labels = label_encoder.inverse_transform(test_predictions_labels)

# Save final test predictions
results_test_df = pd.DataFrame({'id': id_test, 'subreddit': test_predictions_labels})
results_test_df.to_csv("final_predictions_dnn_test.csv", index=False)
print("\nFinal test predictions saved to final_predictions_dnn_test.csv")


Hyperparameter tuning...

Testing parameters: {'batch_size': 32, 'dropout_rate': 0.2, 'layer_sizes': (512, 256, 128), 'learning_rate': 0.001}
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Parameters: {'batch_size': 32, 'dropout_rate': 0.2, 'layer_sizes': (512, 256, 128), 'learning_rate': 0.001}, Average Accuracy = 0.6643
Testing parameters: {'batch_size': 32, 'dropout_rate': 0.2, 'layer_sizes': (512, 256, 128), 'learning_rate': 0.0001}
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
Parameters: {'batch_size': 32, 'dropout_rate': 0.2, 'layer_sizes': (512, 256, 128), 'learning_rate': 0.0001}, Average Accuracy = 0.6693
Testing parameters: {'batch_size': 32, 'dropout_rate': 0.2, 'layer_sizes': (256, 128, 64), 'learning_rate': 0.001}
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/